In [1]:
!pip install -q tensorflow
!pip install -q datasets
!pip install -q accelerate
!pip install -q transformers
!pip install -q emoji
!pip install -q tf-keras

In [2]:
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
sys.path.append('..')
tf.__version__

'2.19.0'

In [3]:
# Will result in error, ignore it

# if tf.config.list_physical_devices('GPU'):
#     physical_devices = tf.config.list_physical_devices('GPU')
#     tf.config.experimental.set_memory_growth(physical_devices[0], True)
#     print('Using GPU:', tf.test.gpu_device_name())
#     !nvcc -V
# else: raise ValueError('Running on CPU is not recommended.')

In [4]:
# Skip this step

# !git clone https://github.com/khamdd/absa-2025
# %cd ./absa-2025
# !mkdir predictions
# !ls

# Constants Setup

In [6]:
TRAIN_PATH = '../datasets/eval_2014/Restaurants_Train_v2.xml'
PREPROCESSED_TRAIN_PATH = '../datasets/eval_2014/Restaurants_Train_v2.csv'
PRETRAINED_MODEL = 'bert-base-uncased'
MODEL_NAME = "Restaurant-v1"
MAX_LENGTH = 256
BATCH_SIZE = 21
EPOCHS = 20

In [7]:
from processors.eval2014_processor import Eval2014Loader

Eval2014Loader.xmlToCSV(TRAIN_PATH, PREPROCESSED_TRAIN_PATH)

CSV file generated:../datasets/eval_2014/Restaurants_Train_v2.csv


In [8]:
raw_datasets = Eval2014Loader.load(PREPROCESSED_TRAIN_PATH)
raw_datasets

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Review', 'ambience', 'anecdotes/miscellaneous', 'food', 'price', 'service'],
        num_rows: 3041
    })
})

# Preprocess and Tokenize data

In [10]:
from processors.english_processor import EnglishTextPreprocessor

eng_preprocessor = EnglishTextPreprocessor()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dangd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dangd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dangd\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
tokens = tokenizer.encode("This is a auto tokenizer test string")
print('Encode:', tokens, '\nDecode:', tokenizer.decode(tokens))
tokenizer.model_input_names

Encode: [101, 2023, 2003, 1037, 8285, 19204, 17629, 3231, 5164, 102] 
Decode: [CLS] this is a auto tokenizer test string [SEP]


['input_ids', 'token_type_ids', 'attention_mask']

In [12]:
preprocessed_datasets = Eval2014Loader.preprocess_and_tokenize(raw_datasets, eng_preprocessor, tokenizer, BATCH_SIZE * 2, MAX_LENGTH)
preprocessed_datasets.save_to_disk('../datasets/preprocessed_restaurant')
display(preprocessed_datasets)
pd.DataFrame({
    'raw_datasets': raw_datasets['train']['Review'][:10],
    'encoded_input_ids': preprocessed_datasets['train']['input_ids'][:10],
    'decoded_input_ids': [tokenizer.decode(preprocessed_datasets['train'][i]['input_ids']) for i in range(10)]
})

[INFO] Tokenizing text data...


Map:   0%|          | 0/3041 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3041 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['ambience', 'anecdotes/miscellaneous', 'food', 'price', 'service', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3041
    })
})

,raw_datasets,encoded_input_ids,decoded_input_ids
0,But the staff was so horrible to us.,"[101, 2021, 1996, 3095, 11333, 2061, 9202, 200...",[CLS] but the staff wa so horrible to u [SEP] ...
1,"To be completely fair, the only redeeming fact...","[101, 2000, 2022, 3294, 4189, 1996, 2069, 2417...",[CLS] to be completely fair the only redeeming...
2,"The food is uniformly exceptional, with a very...","[101, 1996, 2833, 2003, 27423, 11813, 2007, 10...",[CLS] the food is uniformly exceptional with a...
3,Where Gabriela personaly greets you and recomm...,"[101, 2073, 6127, 2050, 3167, 2100, 17021, 201...",[CLS] where gabriela personaly greets you and ...
4,"For those that go once and don't enjoy it, all...","[101, 2005, 2216, 2008, 2175, 2320, 1998, 2123...",[CLS] for those that go once and dont enjoy it...
5,"Not only was the food outstanding, but the lit...","[101, 2025, 2069, 11333, 1996, 2833, 5151, 202...",[CLS] not only wa the food outstanding but the...
6,It is very overpriced and not very tasty.,"[101, 2009, 2003, 2200, 2058, 18098, 6610, 209...",[CLS] it is very overpriced and not very tasty...
7,Our agreed favorite is the orrechiete with sau...,"[101, 2256, 3530, 5440, 2003, 1996, 26914, 159...",[CLS] our agreed favorite is the orrechiete wi...
8,The Bagels have an outstanding taste with a te...,"[101, 1996, 4524, 2884, 2031, 2019, 5151, 5510...",[CLS] the bagel have an outstanding taste with...
9,Nevertheless the food itself is pretty good.,"[101, 6600, 1996, 2833, 2993, 2003, 3492, 2204...",[CLS] nevertheless the food itself is pretty g...


In [13]:
from datasets import load_from_disk
preprocessed_datasets = load_from_disk('../datasets/preprocessed_restaurant')
preprocessed_datasets = Eval2014Loader.labels_to_flatten_onehot(preprocessed_datasets)
preprocessed_datasets

[INFO] Transforming "Aspect#Categoy,Polarity" labels to flattened one-hot encoding...


Map (num_proc=8):   0%|          | 0/3041 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['FlattenOneHotLabels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3041
    })
})

# Prepare for TensorFlow Training

In [15]:
ASPECT_CATEGORY_NAMES = raw_datasets['train'].column_names[1:]
steps_per_epoch = len(preprocessed_datasets['train']) // BATCH_SIZE
total_steps = EPOCHS * steps_per_epoch
print(ASPECT_CATEGORY_NAMES)

['ambience', 'anecdotes/miscellaneous', 'food', 'price', 'service']


In [16]:
train_tf_dataset = preprocessed_datasets['train'].to_tf_dataset(
    columns=tokenizer.model_input_names, label_cols='FlattenOneHotLabels',
    batch_size=BATCH_SIZE, shuffle=True, num_workers=8
)

In [36]:
from tf_keras.optimizers import Adam
from tf_keras.optimizers.schedules import CosineDecay
from tf_keras.callbacks import EarlyStopping

optimizer = Adam(learning_rate=CosineDecay(
    initial_learning_rate = 1e-4,
    warmup_target = 2e-4,
    warmup_steps = int(total_steps * 0.15), # 15% of total_steps
    decay_steps = int(total_steps * 0.3), # Next 30% of total_steps
    alpha = 0.1, # Minimum lr for decay as a fraction of initial_learning_rate
))

early_stop_callback = EarlyStopping(
    monitor = 'val_loss',
    patience = 3, # Stop if no improvement after 3 epochs
    restore_best_weights = True, # You can obtain HIGHER metrics on the test set with longer training time if you set this to False
    # Because after some experiments, I found that even with higher val_loss, it still results in better metric reports on the test set. 
    # This maybe because the training set and the test set have more similarities than the validation data.
    # But I think this is not fair, as we already have prior knowledge about the test set and we modified our training based on the performance on this set. 
    # In real-world, we should only modify our training based on the performance on the validation data
    verbose = 1
)

# Fine Tuning with TensorFlow

In [41]:
%%time
from acsa_model import ABSA2025MultiTask
from helper import plot_training_history
model = ABSA2025MultiTask(PRETRAINED_MODEL, ASPECT_CATEGORY_NAMES, optimizer, name=MODEL_NAME)

history = model.fit(
    train_tf_dataset,
    # validation_data = val_tf_dataset,
    callbacks = [early_stop_callback],
    epochs = EPOCHS,
    verbose = 1
).history

model.save_weights(f'./weights/{MODEL_NAME}/{MODEL_NAME}', save_format='tf')
plot_training_history(history)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/20

  2/145 [..............................] - ETA: 1:27:48 - loss: 0.4735

KeyboardInterrupt: 